[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

英文原版：https://github.com/langchain-ai/langchain-academy/tree/main

译者：Bryce Wang, brycewang2018@gmail.com

# LangChain Academy

欢迎来到 LangChain 学院！

## 背景

在 LangChain，我们的目标是让构建大型语言模型（LLM）应用变得简单。LLM 应用的一种形式是代理（agent）。围绕代理的构建充满了许多令人兴奋的可能性，因为它们能够自动化之前不可能完成的各种任务。

然而，实际上，要构建能够可靠执行这些任务的系统极具挑战性。在与用户合作将代理投入生产的过程中，我们了解到需要更好的控制。例如，可能需要代理优先调用特定工具，或根据状态使用不同的提示词。

为了解决这一问题，我们开发了 [LangGraph](https://langchain-ai.github.io/langgraph/) ——一个用于构建代理和多代理应用的框架。LangGraph 独立于 LangChain 包，其核心设计理念是帮助开发人员在代理工作流中添加更好的精确性和控制能力，适合应对真实世界系统的复杂性。

## 课程结构

课程分为若干模块，每个模块专注于 LangGraph 相关的特定主题。每个模块都有一个文件夹，里面包含一系列笔记本文件。每个笔记本都会配有一个视频帮助讲解概念，但笔记本内容本身也是独立的，可以脱离视频进行学习。每个模块文件夹还包含一个 studio 文件夹，其中有一些可以加载到 LangGraph Studio 的图表，LangGraph Studio 是我们专门为构建 LangGraph 应用提供的 IDE。（译者注：为了简化学习，中文版本不涉及 LangGraph Studio 和 .env 的任何内容，代码全部在 notebook 中运行。）


## Setup

在开始之前，请按照 README 文件中的说明创建环境并安装依赖。

## Chat models

在本课程中，我们将使用 聊天模型，它们以一系列消息作为输入，并输出聊天消息。LangChain 本身不托管任何聊天模型，而是依赖第三方集成。这里列出了 LangChain 支持的第三方聊天模型集成！默认情况下，本课程将使用 ChatOpenAI，因为它既流行又高效。如前所述，请确保您拥有 OPENAI_API_KEY。

我们将检查您的 OPENAI_API_KEY 是否已设置，如果未设置，您将被要求输入（译者注：为了简化操作，本译版全部使用 getpass 进行  api key 的录入）。



In [2]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [3]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

[这里](https://python.langchain.com/v0.2/docs/how_to/#chat-models)是关于聊天模型可以做什么的实用指南，但我们会在下面展示一些亮点。如果您已按照 `README` 中的说明运行了 `pip install -r requirements.txt`，那么您已经安装了 `langchain-openai` 包。借助它，我们可以实例化我们的 `ChatOpenAI` 模型对象。如果您是首次注册 API，您应该会收到可用于任何模型的[免费额度](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517)。您可以在这里查看各种模型的[定价](https://openai.com/api/pricing/)。笔记本将默认使用 gpt-4o，因为它在质量、价格和速度之间具有良好的平衡[详情见此](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini)，但您也可以选择价格较低的 `gpt-3.5` 系列模型。

聊天模型有一些标准参数，其中两个最常见的是：

- `model`：模型名称
- `temperature`：采样温度

`temperature` 控制模型输出的随机性或创造性，接近 `0` 的低温会生成更具确定性和专注性的输出，适用于需要准确性或事实响应的任务。接近 `1` 的高温适合创意任务或生成多样化的响应。

In [4]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

LangChain 的 ChatModels 聊天模型提供了一些[默认方法](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface)。我们主要会使用：

- `stream`：流式返回响应块 stream back chunks of the response
- `invoke`：对输入调用链 call the chain on an input

此外，聊天模型以消息[messages](https://python.langchain.com/v0.2/docs/concepts/#messages)作为输入。消息具有角色（描述谁在发送消息）和内容属性。我们将在后面详细讨论这一点，但这里先展示基础内容。

In [5]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Bryce")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-ff480d22-d098-4061-a9bc-f2db40775ca6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

我们获得了一个 `AIMessage` 响应。此外，请注意我们可以直接用字符串调用聊天模型。当输入是字符串时，它会被转换为 `HumanMessage` 并传递给底层模型。

In [5]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None}, id='run-c6736fc6-825a-4326-bf21-cb458cd0a47a-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
gpt35_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ee1a8947-826d-499a-84ab-fc97468cc188-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

这种接口在所有聊天模型中保持一致，通常会在每个笔记本启动时初始化一次模型。

因此，如果您更喜欢其他提供商的模型，可以轻松切换而无需更改下游代码。

In [6]:
_set_env("TAVILY_API_KEY")

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is LangGraph?")

In [8]:
search_docs

[{'url': 'https://langchain-ai.github.io/langgraph/',
  'content': 'Overview¶. LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures, differentiating it from DAG-based solutions.'},
 {'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'content': 'LangGraph is a library within the LangChain ecosystem designed to tackle these challenges head-on. LangGraph provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner.'},
 {'url': 'https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141',
  'content': 'LangGraph is a versatile tool for building complex, stateful applications with LLMs. By understanding its core concep

In [ ]:
# END